---

# <center> GFootball Academy </center>

---
<center><a data-flickr-embed="true" href="https://www.flickr.com/photos/robocup2013/9175658500/in/photostream/" title="BvOF RoboCup2013 - finals humanoid kid size soccer"><img src="https://live.staticflickr.com/5482/9175658500_76331391bf_c.jpg" width="800" height="533" alt="BvOF RoboCup2013 - finals humanoid kid size soccer"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script></center>

<center><small>Photo credit <a href="https://www.flickr.com/photos/robocup2013/">RoboCup2013</a></small></center>

---

# Start Install
Install kaggle-environments and gfootball dependencies, clone gfootball source and download football engine binary.

In [ ]:
%%bash
git clone -q https://github.com/Kaggle/kaggle-environments.git
cd kaggle-environments && pip3 install -qq . && cd ..

apt-get -qq update
apt-get -qq install libsdl2-gfx-dev libsdl2-ttf-dev > /dev/null

GRF_VER=v2.7
GRF_PATH=football/third_party/gfootball_engine/lib
GRF_URL=https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_${GRF_VER}.so
git clone -q -b ${GRF_VER} https://github.com/google-research/football.git
mkdir -p ${GRF_PATH}
wget -q ${GRF_URL} -O ${GRF_PATH}/prebuilt_gameplayfootball.so

# Add a Custom Scenario
Include a custom scenario inside the gfootball source. May be useful for quick bot testing or training. More example scenarios in the [repository](https://github.com/google-research/football/tree/master/gfootball/scenarios).

In [ ]:
%%writefile football/gfootball/scenarios/academy_custom.py
from . import *

def build_scenario(builder):
    builder.config().game_duration = 300
    builder.config().deterministic = False
    builder.config().offsides = False
    builder.config().end_episode_on_score = True
    builder.config().end_episode_on_out_of_play = True
    builder.config().end_episode_on_possession_change = True
    builder.SetBallPosition(0.3, -0.05)

    builder.SetTeam(Team.e_Left)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK)
    builder.AddPlayer(0.1, -0.1, e_PlayerRole_CF)

    builder.SetTeam(Team.e_Right)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK)
    builder.AddPlayer(-0.7, 0.05, e_PlayerRole_CB)

# Finish Install
Complete gfootball installation with the custom scenario included.

In [ ]:
%%bash
cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install -qq . && cd ..
rm -rf football kaggle-environments

After installation, scenario files can be modified or added in the install location.

In [ ]:
!find / -name academy*.py

---
# Modified Template Bot
A modified version of the [Template Bot](https://www.kaggle.com/piotrstanczyk/gfootball-template-bot) with some ideas from [GFootball Rules from Environment Exploration](https://www.kaggle.com/sx2154/gfootball-rules-from-environment-exploration) and [Simple Baseline Bot](https://www.kaggle.com/eugenkeil/simple-baseline-bot).

In [ ]:
%%writefile submission.py
from math import sqrt, atan2, pi
from kaggle_environments.envs.football.helpers import *

def angle(src, tgt):
    xdir = tgt[0] - src[0]
    ydir = tgt[1] - src[1]
    theta = round(atan2(xdir, -ydir) * 180 / pi, 2)
    while theta < 0:
        theta += 360
    return theta


def direction(src, tgt):
    actions = [Action.Top, Action.TopRight, Action.Right, Action.BottomRight, 
               Action.Bottom, Action.BottomLeft, Action.Left, Action.TopLeft]
    theta = angle(src, tgt)
    index = int(((theta+45/2)%360)/45)
    return actions[index]

@human_readable_agent
def agent(obs):
    goal_pos = [1, 0]
    ball_pos = obs["ball"]
    
    player_pos = obs["left_team"][obs["active"]]
    player_x, player_y = player_pos
    
    ball_owned = (obs["ball_owned_team"] == 0 and 
                  obs["ball_owned_player"] == obs["active"])

    def shot(shot_dir):
        if shot_dir not in obs["sticky_actions"]:
            return shot_dir
        return Action.Shot
    
    def high_pass(pass_dir):
        if pass_dir not in obs["sticky_actions"]:
            return pass_dir
        return Action.HighPass   
    
    if  player_x < 0.6:
        if Action.Sprint not in obs['sticky_actions']:
            return Action.Sprint
    else:
        if Action.Sprint in obs['sticky_actions'] and ball_owned:
            return Action.ReleaseSprint
    
    if ball_owned:
        goal_dir = direction(player_pos, goal_pos)
        if player_x < -0.6:
            return shot(goal_dir)
        if player_x < -0.4:
            return high_pass(goal_dir)
        if player_x > 0.6:
            return shot(goal_dir)
        if player_x > 0.4 and abs(player_y) < 0.2:
            return shot(goal_dir)
        return goal_dir
        
    return direction(player_pos, ball_pos)

---
# Academy Scenarios

In [ ]:
from kaggle_environments import make

scenarios = {0: "academy_custom",
             1: "academy_3_vs_1_with_keeper", 
             2: "academy_corner", 
             3: "academy_counterattack_easy", 
             4: "academy_counterattack_hard", 
             5: "academy_empty_goal", 
             6: "academy_empty_goal_close", 
             7: "academy_pass_and_shoot_with_keeper", 
             8: "academy_run_pass_and_shoot_with_keeper", 
             9: "academy_run_to_score", 
             10: "academy_run_to_score_with_keeper",
             11: "academy_single_goal_versus_lazy",
             12: "11_vs_11_kaggle"}

scenario_num = 0
env = make("football", debug=True,
           configuration={"save_video": True, 
                          "scenario_name": scenarios[scenario_num], 
                          "running_in_notebook": True})
         
output = env.run(["submission.py", "run_right"])

scores = output[-1][0]["observation"]["players_raw"][0]["score"]
print("Scores  {0} : {1}".format(*scores))
print("Rewards {0} : {1}".format(output[-1][0]["reward"], output[-1][1]["reward"]))

---
# Visualization

This animation is created using the beautiful tutorial from [Human Readable Visualization](https://www.kaggle.com/jaronmichal/human-readable-visualization).


Modifications made to the original: 
1. Added player actions.
2. Added markers for active players on both teams.
3. Changed relative sizes of pitch, goal posts and players.
4. Various cosmetic enhancements to fonts and colours.

In [ ]:
%%writefile visualizer.py
from matplotlib import animation, patches, rcParams
from matplotlib import pyplot as plt
from kaggle_environments.envs.football.helpers import *

WIDTH = 110
HEIGHT = 46.2
PADDING = 10


def initFigure(figwidth=12):
    figheight = figwidth * (HEIGHT + 2 * PADDING) / (WIDTH + 2 * PADDING)

    fig = plt.figure(figsize=(figwidth, figheight))
    ax = plt.axes(xlim=(-PADDING, WIDTH + PADDING), ylim=(-PADDING, HEIGHT + PADDING))
    plt.axis("off")
    return fig, ax


def drawPitch(ax):
    paint = "white"

    # Grass around pitch
    rect = patches.Rectangle((-PADDING / 2, -PADDING / 2), WIDTH + PADDING, HEIGHT + PADDING,
                             lw=1, ec="black", fc="#3f995b", capstyle="round")
    ax.add_patch(rect)

    # Pitch boundaries
    rect = plt.Rectangle((0, 0), WIDTH, HEIGHT, ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Middle line
    plt.plot([WIDTH / 2, WIDTH / 2], [0, HEIGHT], color=paint, lw=2)

    # Dots
    dots_x = [11, WIDTH / 2, WIDTH - 11]
    for x in dots_x:
        plt.plot(x, HEIGHT / 2, "o", color=paint, lw=2)

    # Penalty box
    penalty_box_dim = [16.5, 40.3]
    penalty_box_pos_y = (HEIGHT - penalty_box_dim[1]) / 2

    rect = plt.Rectangle((0, penalty_box_pos_y),
                         penalty_box_dim[0], penalty_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, penalty_box_pos_y), -
                         penalty_box_dim[0], penalty_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Goal box
    goal_box_dim = [5.5, penalty_box_dim[1] - 11 * 2]
    goal_box_pos_y = (penalty_box_pos_y + 11)

    rect = plt.Rectangle((0, goal_box_pos_y),
                         goal_box_dim[0], goal_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, goal_box_pos_y),
                         -goal_box_dim[0], goal_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Goals
    goal_width = 0.044 / 0.42 * HEIGHT
    goal_pos_y = (HEIGHT / 2 - goal_width / 2)
    rect = plt.Rectangle((0, goal_pos_y), -2, goal_width,
                         ec=paint, fc=paint, lw=2, alpha=0.3)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, goal_pos_y), 2, goal_width,
                         ec=paint, fc=paint, lw=2, alpha=0.3)
    ax.add_patch(rect)

    # Middle circle
    mid_circle = plt.Circle([WIDTH / 2, HEIGHT / 2], 9.15, color=paint, fc="None", lw=2)
    ax.add_artist(mid_circle)

    # Penalty box arcs
    left = patches.Arc([11, HEIGHT / 2], 2 * 9.15, 2 * 9.15,
                       color=paint, fc="None", lw=2, angle=0, theta1=308, theta2=52)
    ax.add_patch(left)
    right = patches.Arc([WIDTH - 11, HEIGHT / 2], 2 * 9.15, 2 * 9.15,
                        color=paint, fc="None", lw=2, angle=180, theta1=308, theta2=52)
    ax.add_patch(right)

    # Arcs on corners
    corners = [[0, 0], [WIDTH, 0], [WIDTH, HEIGHT], [0, HEIGHT]]
    angle = 0
    for x, y in corners:
        c = patches.Arc([x, y], 2, 2,
                        color=paint, fc="None", lw=2, angle=angle, theta1=0, theta2=90)
        ax.add_patch(c)
        angle += 90


def scale_x(x):
    return (x + 1) * (WIDTH / 2)


def scale_y(y):
    return (y + 0.42) * (HEIGHT / 0.42 / 2)


def extract_data(raw_obs):
    obs = raw_obs[0]["observation"]["players_raw"][0]
    res = dict()
    res["left_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["left_team"]]
    res["right_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["right_team"]]

    ball_x, ball_y, ball_z = obs["ball"]
    res["ball"] = [scale_x(ball_x), scale_y(ball_y), ball_z]
    res["score"] = obs["score"]
    res["steps_left"] = obs["steps_left"]
    res["ball_owned_team"] = obs["ball_owned_team"]

    left_active = raw_obs[0]["observation"]["players_raw"][0]["active"]
    res["left_player"] = res["left_team"][left_active]

    right_active = raw_obs[1]["observation"]["players_raw"][0]["active"]
    res["right_player"] = res["right_team"][right_active]

    res["right_team_roles"] = obs["right_team_roles"]
    res["left_team_roles"] = obs["left_team_roles"]
    res["left_team_direction"] = obs["left_team_direction"]
    res["right_team_direction"] = obs["right_team_direction"]
    res["game_mode"] = GameMode(obs["game_mode"]).name
    return res


def draw_team(obs, team, side):
    x_coords, y_coords = zip(*obs[side])
    team.set_data(x_coords, y_coords)


def draw_ball(obs, ball):
    ball.set_markersize(8 + obs["ball"][2])  # Scale size of ball based on height
    ball.set_data(obs["ball"][:2])


def draw_active_players(obs, left_player, right_player):
    x1, y1 = obs["left_player"]
    left_player.set_data(x1, y1)

    x2, y2 = obs["right_player"]
    right_player.set_data(x2, y2)

    if obs["ball_owned_team"] == 0:
        left_player.set_markerfacecolor("yellow")
        left_player.set_markersize(20)
        right_player.set_markerfacecolor("blue")
        right_player.set_markersize(18)
    elif obs["ball_owned_team"] == 1:
        left_player.set_markerfacecolor("firebrick")
        left_player.set_markersize(18)
        right_player.set_markerfacecolor("yellow")
        right_player.set_markersize(20)
    else:
        left_player.set_markerfacecolor("firebrick")
        left_player.set_markersize(18)
        right_player.set_markerfacecolor("blue")
        right_player.set_markersize(18)


def draw_team_active(obs, team_left_active, team_right_active):
    team_left_active.set_data(WIDTH / 2 - 7, -7)
    team_right_active.set_data(WIDTH / 2 + 7, -7)

    if obs["ball_owned_team"] == 0:
        team_left_active.set_markerfacecolor("indianred")
    else:
        team_left_active.set_markerfacecolor("mistyrose")

    if obs["ball_owned_team"] == 1:
        team_right_active.set_markerfacecolor("royalblue")
    else:
        team_right_active.set_markerfacecolor("lightcyan")


def draw_players_directions(obs, directions, side):
    index = 0
    if "right" in side:
        index = 11
    for i, player_dir in enumerate(obs[f"{side}_direction"]):
        x_dir, y_dir = player_dir
        dist = (x_dir ** 2 + y_dir ** 2)**0.5 + 0.00001  # to prevent division by 0
        x = obs[side][i][0]
        y = obs[side][i][1]
        directions[i + index].set_data([x, x + x_dir / dist], [y, y + y_dir / dist])


def player_actions(step, side):
    if side == 0:
        actions = {0: "idle", 1: "←", 2: "↖", 3: "↑", 4: "↗", 5: "→", 6: "↘", 7: "↓", 8: "↙",
                   9: "l_pass", 10: "h_pass", 11: "s_pass", 12: "shot",
                   13: "sprint", 14: "rel_dir", 15: "rel_spr",
                   16: "slide", 17: "dribble", 18: "rel_dri"}
    else:
        actions = {0: "idle", 1: "→", 2: "↘", 3: "↓", 4: "↙", 5: "←", 6: "↖", 7: "↑", 8: "↗",
                   9: "l_pass", 10: "h_pass", 11: "s_pass", 12: "shot",
                   13: "sprint", 14: "rel_dir", 15: "rel_spr",
                   16: "slide", 17: "dribble", 18: "rel_dri"}

    obs = step[side]["observation"]["players_raw"][0]

    if obs["sticky_actions"][8]:
        spr = "+spr"
    else:
        spr = "-spr"

    if obs["sticky_actions"][9]:
        dri = "+dri"
    else:
        dri = "-dri"

    if 1 in obs["sticky_actions"][0:8]:
        i = obs["sticky_actions"][0:8].index(1) + 1
        drn = actions[i]
    else:
        drn = "|"

    if step[side]["action"]:
        act = actions[step[side]["action"][0]]
    else:
        act = "idle"

    return f"{spr} {dri} {drn} [{act}]".ljust(24, " ")


steps = None
drawings = None
directions = None
ball = left_player = right_player = None
team_left = team_right = None
team_left_active = team_right_active = None
team_left_actions = team_right_actions = None
team_left_number = team_right_number = None
team_left_direction = team_right_direction = None
text_frame = game_mode = match_info = None


def init():
    ball.set_data([], [])
    left_player.set_data([], [])
    right_player.set_data([], [])
    team_left.set_data([], [])
    team_right.set_data([], [])
    team_left_active.set_data([], [])
    team_right_active.set_data([], [])
    return drawings


def animate(i):
    obs = extract_data(steps[i])

    # Draw info about ball possesion
    draw_active_players(obs, left_player, right_player)
    draw_team_active(obs, team_left_active, team_right_active)

    # Draw players
    draw_team(obs, team_left, "left_team")
    draw_team(obs, team_right, "right_team")

    draw_players_directions(obs, directions, "left_team")
    draw_players_directions(obs, directions, "right_team")

    draw_ball(obs, ball)

    # Draw textual informations
    text_frame.set_text(f"Step {i}/{obs['steps_left'] + i - 1}")
    game_mode.set_text(f"{obs['game_mode']} Mode")

    score_a, score_b = obs["score"]
    match_info.set_text(f"{score_a} : {score_b}")

    team_left_actions.set_text(player_actions(steps[i], 0))
    team_right_actions.set_text(player_actions(steps[i], 1))

    team_left_number.set_text(str(steps[i][0]["observation"]["players_raw"][0]["active"]))
    team_right_number.set_text(str(steps[i][1]["observation"]["players_raw"][0]["active"]))

    return drawings


def visualize(trace):
    global steps
    global drawings
    global directions
    global ball, left_player, right_player
    global team_left, team_right
    global team_left_active, team_right_active
    global text_frame, game_mode, match_info
    global team_left_actions, team_right_actions
    global team_left_number, team_right_number
    global team_left_direction, team_right_direction

    rcParams['font.family'] = 'monospace'
    rcParams['font.size'] = 12

    steps = trace

    fig, ax = initFigure()
    drawPitch(ax)
    ax.invert_yaxis()

    left_player, = ax.plot([], [], "o", ms=18, mfc="firebrick", mew=0, alpha=0.5)
    right_player, = ax.plot([], [], "o", ms=18, mfc="blue", mew=0, alpha=0.5)
    team_left, = ax.plot([], [], "o", ms=12, mfc="firebrick", mew=1, mec="white")
    team_right, = ax.plot([], [], "o", ms=12, mfc="blue", mew=1, mec="white")
    ball, = ax.plot([], [], "o", ms=8, mfc="wheat", mew=1, mec="black")

    team_left_active, = ax.plot([], [], "o", ms=16, mfc="mistyrose", mec="None")
    team_right_active, = ax.plot([], [], "o", ms=16, mfc="lightcyan", mec="None")

    textheight = -6
    text_frame = ax.text(-5, textheight, "", ha="left")
    match_info = ax.text(WIDTH / 2, textheight, "", ha="center", fontweight="bold")
    game_mode = ax.text(WIDTH + 5, textheight, "", ha="right")

    team_left_actions = ax.text(WIDTH / 4 + 2, textheight, "", ha="center")
    team_right_actions = ax.text(3 * WIDTH / 4 + 2, textheight, "", ha="center")

    team_left_number = ax.text(WIDTH / 2 - 7, -6.3, "", ha="center", fontsize=10)
    team_right_number = ax.text(WIDTH / 2 + 7, -6.3, "", ha="center", fontsize=10)

    # Drawing of directions definitely can be done in a better way
    directions = []
    for _ in range(22):
        direction, = ax.plot([], [], color="yellow", lw=1.5)
        directions.append(direction)

    drawings = [team_left_active, team_right_active, left_player, right_player,
                team_left, team_right, ball, text_frame, match_info,
                game_mode, team_left_actions, team_right_actions, team_left_number, team_right_number]

    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
    anim = animation.FuncAnimation(fig, animate, init_func=init, blit=True,
                                   interval=100, frames=len(steps), repeat=True)
    return anim

In [ ]:
from IPython.display import HTML
from visualizer import visualize
viz = visualize(output)

In [ ]:
HTML(viz.to_html5_video())